📌 week5 내용 주차에 해당되는 과제는 Glove 모델 실습, NER task 실습, Dependency Parsing task 실습으로 구성되어 있습니다. (**참고** : **제출은 week6 branch 복습과제로!**)

📌 위키독스의 딥러닝을 이용한 자연어 처리 입문 교재 실습, 캐글 노트북 등의 자료로 구성되어있는 과제입니다. 

📌 안내된 링크에 맞추어 **직접 코드를 따라 치면서 (필사)** 해당 nlp task 의 기본적인 라이브러리와 메서드를 숙지해보시면 좋을 것 같습니다😊 필수라고 체크한 부분은 과제에 반드시 포함시켜주시고, 선택으로 체크한 부분은 자율적으로 스터디 하시면 됩니다.

📌 궁금한 사항은 깃허브 이슈나, 카톡방, 세션 발표 시작 이전 시간 등을 활용하여 자유롭게 공유해주세요!

In [ ]:
import nltk
# nltk colab 환경에서 실행시 필요한 코드입니다. 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### 1️⃣ **Glove**



👀 **내용 복습** 
* 스탠포드 대학에서 개발한 카운트 기반과 예측 기반을 모두 사용하는 단어 임베딩 방법론 
* word2vec 의 단점을 보완해서 나온 모델 
* glove model 의 **input 은 반드시 동시등장행렬 형태**여야 한다 ⭐

![1](https://www.dropbox.com/s/nz0ji4yzre56ifv/word_presentation.png?raw=1) 




🤔 한국어 예제는 없는 것 같습니다. 논문에서는 k-Glove 로 소개되는 연구가 있긴 한데, 좀 더 알아봐야 할 것 같아요!

➕ [논문1](https://scienceon.kisti.re.kr/srch/selectPORSrchArticle.do?cn=NPAP13255003&dbt=NPAP)


➕[논문2](https://scienceon.kisti.re.kr/commons/util/originalView.do?cn=CFKO201832073078664&oCn=NPAP13255064&dbt=CFKO&journal=NPRO00383361&keyword=%ED%95%9C%EA%B5%AD%EC%96%B4%20%EB%8C%80%ED%99%94%20%EC%97%94%EC%A7%84%EC%97%90%EC%84%9C%EC%9D%98%20%EB%AC%B8%EC%9E%A5%EB%B6%84%EB%A5%98)

🔹 **1-(1)** glove python

* [실습 : basic code](https://wikidocs.net/22885) 👉 필수

In [ ]:
pip install glove_python_binary


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from glove import Corpus, Glove

result = []

corpus = Corpus()

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬을 생성
corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

#학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


🔹 **1-(2)** pre-trained glove 

* **사전학습모델** : 임의의 값으로 초기화하던 모델의 가중치들을 다른 문제에 학습시킨 가중치들로 초기화하는 방법이다.사전 학습한 가중치를 활용해 학습하고자 하는 본래 문제를 하위문제라고 한다. 

* [실습 : 문장의 긍부정을 판단하는 감성 분류 모델 만들기](https://wikidocs.net/33793) 👉 필수
  * [설명참고](https://omicro03.medium.com/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-16%EC%9D%BC%EC%B0%A8-pre-trained-word-embedding-bb30db424a35)
* pre-trained data 를 가져오는데 시간이 오래걸림
* kaggle 대회에서 주로 이 방식을 많이 사용함
  * [참고](https://lsjsj92.tistory.com/455)

In [ ]:
vocab_size = 20000
output_dim = 128
input_length = 500

v = Embedding(vocab_size, output_dim, input_length=input_length)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print('단어 집합 :',vocab_size)

단어 집합 : 16


In [ ]:
X_encoded = tokenizer.texts_to_sequences(sentences)
print('정수 인코딩 결과 :',X_encoded)

정수 인코딩 결과 : [[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [ ]:
max_len = max(len(l) for l in X_encoded)
print('최대 길이 :',max_len)

최대 길이 : 4


In [ ]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print('패딩 결과 :')
print(X_train)

패딩 결과 :
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim = 4

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Train on 7 samples
Epoch 1/100
7/7 - 0s - loss: 0.6916 - acc: 0.4286 - 53ms/epoch - 8ms/sample
Epoch 2/100
7/7 - 0s - loss: 0.6903 - acc: 0.4286 - 10ms/epoch - 1ms/sample
Epoch 3/100
7/7 - 0s - loss: 0.6889 - acc: 0.5714 - 5ms/epoch - 752us/sample
Epoch 4/100
7/7 - 0s - loss: 0.6876 - acc: 0.7143 - 6ms/epoch - 911us/sample
Epoch 5/100
7/7 - 0s - loss: 0.6862 - acc: 0.7143 - 5ms/epoch - 774us/sample
Epoch 6/100
7/7 - 0s - loss: 0.6849 - acc: 0.7143 - 4ms/epoch - 597us/sample
Epoch 7/100
7/7 - 0s - loss: 0.6835 - acc: 0.7143 - 4ms/epoch - 641us/sample
Epoch 8/100
7/7 - 0s - loss: 0.6821 - acc: 0.7143 - 3ms/epoch - 460us/sample
Epoch 9/100
7/7 - 0s - loss: 0.6808 - acc: 0.7143 - 4ms/epoch - 566us/sample
Epoch 10/100
7/7 - 0s - loss: 0.6794 - acc: 0.7143 - 5ms/epoch - 768us/sample
Epoch 11/100
7/7 - 0s - loss: 0.6780 - acc: 0.7143 - 4ms/epoch - 605us/sample
Epoch 12/100
7/7 - 0s - loss: 0.6767 - acc: 0.7143 - 4ms/epoch - 558us/sample
Epoch 13/100
7/7 - 0s - loss: 0.6753 - acc: 0.7143 - 3ms

In [ ]:
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [ ]:
print(y_train)

[1 0 0 1 1 0 1]


In [ ]:
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall() 
zf.close()

In [ ]:
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]

    # 100개의 값을 가지는 array로 변환
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [ ]:
print(embedding_dict['respectable'])
print('벡터의 차원 수 :',len(embedding_dict['respectable']))

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix))

임베딩 행렬의 크기(shape) : (16, 100)


In [ ]:
print(tokenizer.word_index.items())

dict_items([('nice', 1), ('great', 2), ('best', 3), ('amazing', 4), ('stop', 5), ('lies', 6), ('pitiful', 7), ('nerd', 8), ('excellent', 9), ('work', 10), ('supreme', 11), ('quality', 12), ('bad', 13), ('highly', 14), ('respectable', 15)])


In [ ]:
print('단어 great의 맵핑된 정수 :',tokenizer.word_index['great'])

단어 great의 맵핑된 정수 : 2


In [ ]:
print(embedding_dict['great'])

[-0.013786   0.38216    0.53236    0.15261   -0.29694   -0.20558
 -0.41846   -0.58437   -0.77355   -0.87866   -0.37858   -0.18516
 -0.128     -0.20584   -0.22925   -0.42599    0.3725     0.26077
 -1.0702     0.62916   -0.091469   0.70348   -0.4973    -0.77691
  0.66045    0.09465   -0.44893    0.018917   0.33146   -0.35022
 -0.35789    0.030313   0.22253   -0.23236   -0.19719   -0.0053125
 -0.25848    0.58081   -0.10705   -0.17845   -0.16206    0.087086
  0.63029   -0.76649    0.51619    0.14073    1.019     -0.43136
  0.46138   -0.43585   -0.47568    0.19226    0.36065    0.78987
  0.088945  -2.7814    -0.15366    0.01015    1.1798     0.15168
 -0.050112   1.2626    -0.77527    0.36031    0.95761   -0.11385
  0.28035   -0.02591    0.31246   -0.15424    0.3778    -0.13599
  0.2946    -0.31579    0.42943    0.086969   0.019169  -0.27242
 -0.31696    0.37327    0.61997    0.13889    0.17188    0.30363
 -1.2776     0.044423  -0.52736   -0.88536   -0.19428   -0.61947
 -0.10146   -0.26301  

In [ ]:
for word, index in tokenizer.word_index.items():
    # 단어와 맵핑되는 사전 훈련된 임베딩 벡터값
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value

In [ ]:
embedding_matrix[2]

array([-0.013786  ,  0.38216001,  0.53236002,  0.15261   , -0.29694   ,
       -0.20558   , -0.41846001, -0.58437002, -0.77354997, -0.87866002,
       -0.37858   , -0.18516   , -0.12800001, -0.20584001, -0.22925   ,
       -0.42598999,  0.3725    ,  0.26076999, -1.07019997,  0.62915999,
       -0.091469  ,  0.70348001, -0.4973    , -0.77691001,  0.66044998,
        0.09465   , -0.44893   ,  0.018917  ,  0.33146   , -0.35021999,
       -0.35789001,  0.030313  ,  0.22253001, -0.23236001, -0.19719   ,
       -0.0053125 , -0.25848001,  0.58081001, -0.10705   , -0.17845   ,
       -0.16205999,  0.087086  ,  0.63028997, -0.76648998,  0.51618999,
        0.14072999,  1.01900005, -0.43136001,  0.46138   , -0.43584999,
       -0.47567999,  0.19226   ,  0.36065   ,  0.78987002,  0.088945  ,
       -2.78139997, -0.15366   ,  0.01015   ,  1.17980003,  0.15167999,
       -0.050112  ,  1.26259995, -0.77526999,  0.36030999,  0.95761001,
       -0.11385   ,  0.28035   , -0.02591   ,  0.31246001, -0.15

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

output_dim = 100

model = Sequential()
e = Embedding(vocab_size, output_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.7756 - acc: 0.4286 - 563ms/epoch - 563ms/step
Epoch 2/100
1/1 - 0s - loss: 0.7528 - acc: 0.4286 - 6ms/epoch - 6ms/step
Epoch 3/100
1/1 - 0s - loss: 0.7309 - acc: 0.4286 - 9ms/epoch - 9ms/step
Epoch 4/100
1/1 - 0s - loss: 0.7097 - acc: 0.4286 - 8ms/epoch - 8ms/step
Epoch 5/100
1/1 - 0s - loss: 0.6893 - acc: 0.5714 - 9ms/epoch - 9ms/step
Epoch 6/100
1/1 - 0s - loss: 0.6696 - acc: 0.5714 - 7ms/epoch - 7ms/step
Epoch 7/100
1/1 - 0s - loss: 0.6508 - acc: 0.5714 - 8ms/epoch - 8ms/step
Epoch 8/100
1/1 - 0s - loss: 0.6326 - acc: 0.5714 - 9ms/epoch - 9ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6152 - acc: 0.5714 - 6ms/epoch - 6ms/step
Epoch 10/100
1/1 - 0s - loss: 0.5984 - acc: 0.7143 - 7ms/epoch - 7ms/step
Epoch 11/100
1/1 - 0s - loss: 0.5823 - acc: 0.7143 - 6ms/epoch - 6ms/step
Epoch 12/100
1/1 - 0s - loss: 0.5669 - acc: 0.7143 - 7ms/epoch - 7ms/step
Epoch 13/100
1/1 - 0s - loss: 0.5520 - acc: 0.7143 - 4ms/epoch - 4ms/step
Epoch 14/100
1/1 - 0s - loss: 0.5377 - acc:

🔹 **1-(3)** fine tuning glove
* 미세조정 : 사전 학습한 모든 가중치와 더불어 하위 문제를 위한 최소한의 가중치를 추가해 모델을 추가로 학습하는 방법이다. 

* fine tuning 이 필요한 경우 
  * pretrained model 에 데이터셋에 있는 단어가 포함되지 않은 경우 
  * 데이터 집합이 너무 작아서 전체 모델을 훈련시키기 어려운 경우 

* [Mittens 라이브러리로 fine tuning](https://towardsdatascience.com/fine-tune-glove-embeddings-using-mittens-89b5f3fe4c39) 👉 필수
  *  GloVe 임베딩을 fine-tuning 하기 위한 파이썬 라이브러리
  * [github](https://github.com/roamanalytics/mittens)

* [한국어 소설 텍스트 데이터 미세조정 모델 학습 - GPT2](https://m.blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=horajjan&logNo=222104684132&categoryNo=120&proxyReferer=) 👉 선택 (glove 모델 예제는 아닙니다. fine-tuning 에 초점을 두어서 참고해주시면 좋을 것 같습니다.)

In [ ]:
pip install -U mittens

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
from collections import Counter
from nltk.corpus import brown
from mittens import GloVe, Mittens
from sklearn.feature_extraction import _stop_words
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

* (참고) word2vec pretrained example

➕ [word2vec 사전학습 모델 -한국어1](http://doc.mindscale.kr/km/unstructured/11.html)

➕ [word2vec 사전학습 - 한국어2](https://monetd.github.io/python/nlp/Word-Embedding-Word2Vec-%EC%8B%A4%EC%8A%B5/#%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec-%EB%A7%8C%EB%93%A4%EA%B8%B0)

### **2️⃣ NER**

👀 **내용 복습** 
* 개체명 인식을 사용하면 코퍼스로부터 어떤 단어가 사람, 장소, 조직 등을 의미하는 단어인지를 찾을 수 있다. 




🔹 **2-(1)** NER task by nltk library


* nltk 에서는 개체명 인식기 (NER chunker) 를 지원하고 있다. 
* ne_chunk 는 개체명을 태깅하기 위해서 앞서 품사 태깅 pos_tag 가 수행되어야 한다. 


📌 [basic code](https://wikidocs.net/30682) 👉 필수 

📌 [BIO 표현, LSTM을 활용한 NER 실습](https://wikidocs.net/24682) 👉 선택




In [16]:
#import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [12]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk

sentence = "James is working at Disney in London"

#토큰화 후 품사 태깅

tokenized_sentence = pos_tag(word_tokenize(sentence))
print(tokenized_sentence)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('London', 'NNP')]


In [17]:
#개체명 인식

ner_sentence = ne_chunk(tokenized_sentence)
print(ner_sentence)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE London/NNP))


🔹 **2-(2)** NER task by spacy library


* spaCy 는 자연어처리를 위한 파이썬 기반의 오픈 소스 라이브러리로 다음과 같은 기능을 제공한다. 
  * Tokenization 
  * POS tagging 
  * Lemmatization 
  * Sentence Boundary Detection (SBD)
  * Named Entity Recognition (NER)
  * Similarity
  * Text Classification
  * Rule-based Matching
  * Training
  * Serialization

* spaCy 와 NER
  * .ents → .label_


📌 [basic code](https://frhyme.github.io/python-lib/nlp_spacy_1/) 👉 필수 (NER 부분만)

📌 [kaggle_Custom NER using SpaCy](https://www.kaggle.com/code/amarsharma768/custom-ner-using-spacy/notebook) 👉 선택

  * 훈련되지 않은 데이터 세트에 명명된 엔티티를 학습하는 방법 : 이력서 pdf 데이터 활용 
  * manually labelled 

📌 [한국어 NER](https://github.com/monologg/KoBERT-NER) 👉 참고하면 좋을 자료

➕ [참고](http://aispiration.com/nlp2/nlp-ner-python.html)

In [20]:
import spacy

nlp=spacy.load('en_core_web_sm')

doc = nlp('Apple is looking at buying U.K. startup for $1 billion')

for ent in doc.ents:
  print(ent.text, ent.label_)

Apple ORG
U.K. GPE
$1 billion MONEY


In [21]:
doc = nlp("""But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption.""".replace("\n", " ").strip())

for ent in doc.ents:
  print(ent.text, ent.label_)

Google ORG
Apple’s Siri ORG
iPhones ORG
Amazon ORG
Alexa ORG
Echo GPE
Dot ORG


###**3️⃣ Dependency Parsing**

👀 **내용 복습** 
* 문장의 전체적인 구성/구조 보다는 각 개별단어 간의 '의존관계' 또는 '수식관계' 와 같은 단어간 관계를 파악하는 것이 목적인 NLP Task
* 문장 해석의 모호성을 없애기 위해 Parsing 을 한다.




🔹 **3-(1)** Dependency Parsing by spacy library


* [basic](https://frhyme.github.io/python-lib/nlp_spacy_1/#navigating-parse-tree) 👉 dependecy parsing 부분만 필수
* .dep_ 메서드



In [22]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

noun_chunks = doc.noun_chunks
print(type(noun_chunks))
noun_chunk = list(noun_chunks)[0]
print(type(noun_chunk))
token = noun_chunk[0]
print(type(token))

print("=="*30)
print("""
Text: The original noun chunk text.
Root text: The original text of the word connecting the noun chunk to the rest of the parse.
Root dep: Dependency relation connecting the root to its head.
Root head text: The text of the root token's head.
""".strip())
print("=="*30)
str_format = "{:>25}"*4
for chunk in doc.noun_chunks:
  print(str_format.format(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text))

<class 'generator'>
<class 'spacy.tokens.span.Span'>
<class 'spacy.tokens.token.Token'>
Text: The original noun chunk text.
Root text: The original text of the word connecting the noun chunk to the rest of the parse.
Root dep: Dependency relation connecting the root to its head.
Root head text: The text of the root token's head.
          Autonomous cars                     cars                    nsubj                    shift
      insurance liability                liability                     dobj                    shift
            manufacturers            manufacturers                     pobj                   toward


In [ ]:
#nevigating parse tree

doc=nlp("Autonomous cars shift insurance liability toward manufacurers")
for tok in doc:
  print(tok.text)
  children = list(tok.children)
  print('childre:', children, 'head:', tok.head if tok.head != tok else "!this is root node")
  print("=="*16)

In [26]:
#이를 간단하게 네트워크로 표현

import networkx as nx
import matplotlib.pyplot as plt

nG = nx.Graph()
doc[2] ##root node

def add_n_to_g(inputG, tok):
  inputG.add_node(tok)
  children = list(tok.children)
  if children != []:
    inputG.add_nodes_from(children)
    for c in children:
      inputG.add_edges_from([(tok, c, {'dependency':c.dep_})])
      add_n_to_g(inputG, c)

add_n_to_g(nG, doc[2])
print(nG.nodes(data=True))
print("=="*20)
for e in nG.edges(data=True):
  print(f"{e[0]}, {e[1]}, ### dependency: {e[2]['dependency']}")

[(shift, {}), (cars, {}), (liability, {}), (toward, {}), (Autonomous, {}), (insurance, {}), (manufacturers, {})]
shift, cars, ### dependency: nsubj
shift, liability, ### dependency: dobj
shift, toward, ### dependency: prep
cars, Autonomous, ### dependency: amod
liability, insurance, ### dependency: compound
toward, manufacturers, ### dependency: pobj


🔹 **3-(2)** Spacy (kaggle) 

* 캐글 노트북 환경에서 실습해보는 것을 권장드립니다!

* [kaggle_spaCy](https://www.kaggle.com/code/nirant/hitchhiker-s-guide-to-nlp-in-spacy) 👉 필수
  * 도날드 트럼프 트위터 트윗 내용 데이터 분석


👀 **노트북 키포인트** 
  1. spacy.display 메서드를 사용한 NER 시각화 
  2. Tagging 을 통한 트럼프 트윗 분석 : noun_chunks 는 dependency graph를 고려하여, noun phrase를 뽑아준다. 
  3. [spacy Match](https://yujuwon.tistory.com/entry/spaCy-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-Rule-based-Matching) : 직접 문장/단어 패턴을 등록하여 parsing
  4. Question and answering task using Dependency Parsing
    * spacy display :  ``style = 'dep'``
    * .dep_


In [ ]:
!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
!python -m spacy download en_vectors_web_lg

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
tweets = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/all_djt_tweets.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [37]:
def explain_text_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f'Entity: {ent}, Label: {ent.label_}, {spacy.explain(ent.label_)}')

In [38]:
explain_text_entities(tweets['text'][9])

Entity: Friday, Label: DATE, Absolute or relative dates or periods


In [39]:
one_sentence = tweets['text'][0]
doc = nlp(one_sentence)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [40]:
def redact_names(text):
    doc = nlp(text)
    redacted_sentence = []
    for ent in doc.ents:
        ent.merge()
    for token in doc:
        if token.ent_type_ == "PERSON":
            redacted_sentence.append("[REDACTED]")
        else:
            redacted_sentence.append(token.string)
    return "".join(redacted_sentence)

In [41]:
example_text = tweets['text'][9]
doc = nlp(example_text)
spacy.displacy.render(doc, style='ent', jupyter=True)

for idx, sentence in enumerate(doc.sents):
    for noun in sentence.noun_chunks:
        print(f"sentence {idx+1} has noun chunk '{noun}'")

sentence 1 has noun chunk 'Stock Market'
sentence 1 has noun chunk 'all time'
sentence 1 has noun chunk 'Friday'
sentence 2 has noun chunk 'Congratulations U.S.A.'


In [42]:
one_sentence = tweets['text'][300]
doc = nlp(one_sentence)
spacy.displacy.render(doc, style='ent', jupyter=True)

for token in doc:
    print(token, token.pos_)

Incredibly ADV
beautiful ADJ
ceremony NOUN
as SCONJ
U.S. PROPN
Korean PROPN
War PROPN
remains VERB
are AUX
returned VERB
to ADP
American ADJ
soil NOUN
. PUNCT
Thank VERB
you PRON
to ADP
Honolulu PROPN
and CCONJ
all PRON
of ADP
our PRON
great ADJ
Military ADJ
participants NOUN
on ADP
a DET
job NOUN
well ADV
done VERB
. PUNCT
A DET
special ADJ
thanks NOUN
to ADP
Vice PROPN
President PROPN
Mike PROPN
Pence PROPN
on ADP
delivering VERB
a DET
truly ADV
magnificent ADJ
tribute NOUN
! PUNCT


In [43]:
text = tweets['text'].str.cat(sep=' ')
# spaCy enforces a max limit of 1000000 characters for NER and similar use cases.
# Since `text` might be longer than that, we will slice it off here
max_length = 1000000-1
text = text[:max_length]

# removing URLs and '&amp' substrings using regex
import re
url_reg  = r'[a-z]*[:.]+\S+'
text   = re.sub(url_reg, '', text)
noise_reg = r'\&amp'
text   = re.sub(noise_reg, '', text)

In [44]:
doc = nlp(text)

In [45]:
items_of_interest = list(doc.noun_chunks)
items_of_interest = [str(x) for x in items_of_interest]

In [46]:
trump_topics = []
for token in doc:
    if (not token.is_stop) and (token.pos_ == "NOUN") and (len(str(token))>2):
        trump_topics.append(token)
        
trump_topics = [str(x) for x in trump_topics]

In [47]:
trump_topics = []
for ent in doc.ents:
    if ent.label_ not in ["PERCENT", "CARDINAL", "DATE"]:
#         print(ent.text,ent.label_)
        trump_topics.append(ent.text.strip())